In [44]:
import shutil
import openpyxl
from openpyxl import Workbook, load_workbook
import numpy as np
from openpyxl.chart import ScatterChart, Reference, Series

# Path of the Curve Points exported after each iteration

In [45]:
peak_path=r'C:\Users\kaamil.kaleem\OneDrive - Vibracoustic\New_fixture\TeCo_Dyn_Parameters_Characteriziation_E650\opt_files\dyn_curve\TeCo_Dynamic_NR-E650B-450'

# Path of .inp file (a,b,c values) 

In [46]:
path =r'C:\Users\kaamil.kaleem\OneDrive - Vibracoustic\New_fixture\TeCo_Dyn_Parameters_Characteriziation_E650\opt_files\inp file\TeCo_Dynamic_NR-E650B-450'

# Function to extract a, b, c values from .inp file

In [47]:
def ext_abc(path2):
    f=open(path2)
    inp = f.readlines()
    A=''
    for j in range(1,4):
        for i in range(2, len(inp[j])):
            A+=inp[j][i].replace('\n',', ')
    string = A.strip(', ')
    lst =[float(x) for x in string.split(',')]
    return lst

# Function to extract curve points 

In [48]:
def ext_pp(peak_path2):
    file = open(peak_path2)
    p=file.read()
    v=p.strip('\n')
    k=v.split('\n')
    frq =[]
    d_stf=[]
    for j in range(len(k)):
        val=k[j].split(',')
        frq.append(val[0])
        d_stf.append(val[1])
    return frq, d_stf

# Main part of the code that exports/creates the Excel file

In [49]:
path_excel_2 = r"C:\Users\kaamil.kaleem\OneDrive - Vibracoustic\Desktop\Macro_test_withchart.xlsx"
wb = openpyxl.Workbook()
ws_active=wb.active
#Code to extract a, b, c values 
for i in range(38):
    if i<10:
        path2=path+f"_000{i}.inp"
    else:
        path2=path+f"_00{i}.inp"
    
    lst=ext_abc(path2)
    
    #Creating a new worksheet
    ws_new = wb.create_sheet(f"Sheet{i}")
    
    #Creating the template
    ws_new['A1'] = 'Gmod'
    ws_new['A2'] = 'A'
    ws_new['A3'] = 'B'
    ws_new['A4'] = 'C'
    ws_new['A5'] = 'Static Stifness'
    ws_new['B1'] = 0.56
    ws_new['B5'] = 5.12
    ws_new['D1'] = 'fqr'
    ws_new['E1'] = 'Dyn'
    ws_new['F1'] = 'Norm Frq'
    ws_new['G1'] = 'Norm Dyn'
    ws_new['I1'] = 'Max peak'
    ws_new['I2'] = 'Width peak'
    ws_new['I3'] = 'Frq Peak'
    
    #pasting values in the cells of the new sheet
    ws_new['B2'].value=lst[0]
    ws_new['B3'].value=lst[1]
    ws_new['B4'].value=lst[2]
    if i<10:
        peak_path2=peak_path+f"_000{i}_test.dyn_curve"
    else:
        peak_path2=peak_path+f"_00{i}_test.dyn_curve"
    frq, d_stf =ext_pp(peak_path2)
    f_values= []
    d_stf_values = []
    for j in range(len(frq)):
        ws_new[f'D{j+2}'].value=float(frq[j])
        ws_new[f'E{j+2}'].value=float(d_stf[j])
        ws_new[f'F{j+2}'].value=float(frq[j])/np.sqrt(5.12)
        f_values.append(float(frq[j])/np.sqrt(5.12))
        ws_new[f'G{j+2}'].value=float(d_stf[j])/5.12
        d_stf_values.append(float(d_stf[j])/5.12)
    
wb.save(path_excel_2)

  
        


# To Update the Excel with Graph of the normalized curve

In [51]:
from openpyxl import load_workbook
from openpyxl.chart import ScatterChart, Reference, Series

# Load the existing workbook
workbook = load_workbook(path_excel_2)

# Iterate through each sheet
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]

    # Retrieve the data from the worksheet
    data = []
    for row in sheet.iter_rows(values_only=True):
        data.append(row)

    # Separate the x and y values
    x_values = [row[5] for row in data]  # 6th column, index 5
    y_values = [row[6] for row in data]  # 7th column, index 6

    # Create a scatter chart with the data
    chart = ScatterChart()
    chart.title = "Scatter Plot"

    x_data = Reference(sheet, min_col=6, min_row=2, max_row=sheet.max_row)
    y_data = Reference(sheet, min_col=7, min_row=2, max_row=sheet.max_row)
    series = Series(y_data, x_data)

    chart.series.append(series)

    # Optional: Customize chart properties
    chart.x_axis.title = "X Values"
    chart.y_axis.title = "Y Values"
    chart.style = 13

    # Add the chart to the sheet
    sheet.add_chart(chart, "I10")

# Save the modified workbook
workbook.save(path_excel_2)
